## Cherry Blossom Peak Bloom Prediction - Machine Learning Approach 
* Author: Julia Hsu
* Date: 2/28/2022

In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error, r2_score, mean_absolute_percentage_error, make_scorer

### Load features dataframe

In [2]:
dc_bloom_features = pd.read_csv('../../data/processed_data/features/dc_bloom_features.csv', index_col = 0)
dc_bloom_features.head()

,location,lat,long,alt,year,month,bloom_date,bloom_doy,AGDD_Bloom,AGDD_Mar_13,...,spring_avg_tmin,dec_avg_daily_temp,dec_avg_tmax,dec_avg_tmin,jan_avg_daily_temp,jan_avg_tmax,jan_avg_tmin,feb_avg_daily_temp,feb_avg_tmax,feb_avg_tmin
0,washingtondc,38.88535,-77.038628,0,1980,4,1980-04-06,97,432.2960,294.6138,...,2.379787,5.090883,9.180187,1.001578,1.841934,4.626977,-0.943109,0.534913,4.533650,-3.463823
1,washingtondc,38.88535,-77.038628,0,1981,4,1981-04-03,93,448.8020,309.6097,...,1.022686,2.310882,6.378444,-1.756680,-1.245870,3.150417,-5.642157,4.515485,9.368280,-0.337311
2,washingtondc,38.88535,-77.038628,0,1982,4,1982-04-07,97,424.0151,295.0746,...,1.486407,2.269679,6.528611,-1.989252,-2.381347,2.414614,-7.177309,2.808606,7.532470,-1.915257
3,washingtondc,38.88535,-77.038628,0,1983,4,1983-04-07,97,438.9514,296.0855,...,3.776528,6.869554,11.057188,2.681921,2.521190,6.469740,-1.427361,2.754800,7.595304,-2.085704
4,washingtondc,38.88535,-77.038628,0,1984,4,1984-04-03,94,397.6100,302.0006,...,0.003904,0.623452,4.733088,-3.486184,-1.775054,3.131427,-6.681535,5.305679,10.298952,0.312406


In [3]:
kyoto_bloom_features = pd.read_csv('../../data/processed_data/features/kyoto_bloom_features.csv', index_col = 0)
kyoto_bloom_features.head()

,location,lat,long,alt,year,month,bloom_date,bloom_doy,AGDD_Bloom,AGDD_Mar_13,...,spring_avg_tmin,dec_avg_daily_temp,dec_avg_tmax,dec_avg_tmin,jan_avg_daily_temp,jan_avg_tmax,jan_avg_tmin,feb_avg_daily_temp,feb_avg_tmax,feb_avg_tmin
0,kyoto,35.011983,135.676114,44,1980,4,1980-04-11,102,664.377667,352.2227,...,2.187269,7.155180,11.076990,3.233369,3.541552,6.715957,0.367146,2.629162,6.217429,-0.959104
1,kyoto,35.011983,135.676114,44,1981,4,1981-04-09,99,658.780875,317.4440,...,3.136032,4.503634,8.088251,0.919018,1.252236,4.258996,-1.754524,2.926141,6.344579,-0.492297
2,kyoto,35.011983,135.676114,44,1982,4,1982-04-03,93,625.301695,355.2542,...,3.357173,5.199862,8.904131,1.495593,3.190712,6.582074,-0.200650,3.128944,6.190991,0.066898
3,kyoto,35.011983,135.676114,44,1983,4,1983-04-09,99,764.939470,351.5349,...,3.831513,6.464476,10.192911,2.736041,3.866633,7.310344,0.422921,3.421313,7.056916,-0.214291
4,kyoto,35.011983,135.676114,44,1984,4,1984-04-18,109,683.382862,274.8672,...,0.316914,4.621609,8.045200,1.198017,1.453350,4.250518,-1.343818,1.158684,4.047960,-1.730592


In [4]:
liestal_bloom_features = pd.read_csv('../../data/processed_data/features/liestal_bloom_features.csv', index_col = 0)
liestal_bloom_features.head()

,location,lat,long,alt,year,month,bloom_date,bloom_doy,AGDD_Bloom,AGDD_Mar_13,...,spring_avg_tmin,dec_avg_daily_temp,dec_avg_tmax,dec_avg_tmin,jan_avg_daily_temp,jan_avg_tmax,jan_avg_tmin,feb_avg_daily_temp,feb_avg_tmax,feb_avg_tmin
0,liestal,47.4814,7.730519,350,1980,4,1980-04-13,104,430.072130,326.594896,...,0.953704,2.769104,5.232136,0.306071,-1.466519,0.496612,-3.429650,3.165472,6.465128,-0.134183
1,liestal,47.4814,7.730519,350,1981,4,1981-04-02,92,481.635110,319.989358,...,2.892852,-1.620004,1.272910,-4.512918,-2.855103,-0.637422,-5.072784,-1.754247,1.227642,-4.736136
2,liestal,47.4814,7.730519,350,1982,4,1982-04-18,108,442.937058,318.805830,...,-0.239655,0.113600,1.928978,-1.701778,-0.403605,1.247128,-2.054338,0.047455,3.010888,-2.915977
3,liestal,47.4814,7.730519,350,1983,4,1983-04-11,101,490.948637,314.272555,...,0.961320,2.603456,4.434049,0.772862,2.005489,4.165570,-0.154592,-1.716380,1.151354,-4.584114
4,liestal,47.4814,7.730519,350,1984,4,1984-04-19,110,430.976471,324.000000,...,-2.469351,0.570663,2.977363,-1.836037,0.535159,2.751554,-1.681236,-0.306886,1.991707,-2.605478


### ML model Training 
1. Apply RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor and SVR (linear kernel) ML models
2. Evaluate each model by using 10-fold cross-validation
3. Evaluation metrics: Mean absolute error and Mean Squared Errors (MSE)

In [5]:
# features used for ML model training
used_features = ['lat','alt','winter_avg_daily_temp', 'winter_avg_tmax', 'winter_avg_tmin',
                  'spring_avg_daily_temp', 'spring_avg_tmax', 'spring_avg_tmin',
                  'dec_avg_daily_temp', 'dec_avg_tmax', 'dec_avg_tmin',
                  'jan_avg_daily_temp', 'jan_avg_tmax', 'jan_avg_tmin',
                  'feb_avg_daily_temp', 'feb_avg_tmax', 'feb_avg_tmin',
                  'AGDD_Mar_13']

In [6]:
def pbd_pred_kfold(data, features):
  x = data[features]
  y = data['bloom_doy']
  kf = KFold(n_splits=10)
  models = ['RandomForestRegressor', 'GradientBoostingRegressor','AdaBoostRegressor','SVR_linear']
  metrics = ['train_r2_score','mean_absolute_error','mean_absolute_percentage_error','mae/mean','mean_squared_error','RMSE','max_error','r2_score']
  model_metrics = {model: {metric:0.0 for metric in metrics} for model in models}

  for idx, val in enumerate(kf.split(x)):
      train_i, test_i = val
      y_train, y_test = y[train_i], y[test_i]
      x_train, x_test = x.iloc[train_i], x.iloc[test_i]
      std_scaler = StandardScaler().fit(x)
      x_train = std_scaler.transform(x_train)
      x_test = std_scaler.transform(x_test)
      rf_reg = RandomForestRegressor(random_state=42, n_estimators = 200)
      rf_reg.fit(x_train, y_train)
      rfr_res = rf_reg.predict(x_test)
      rfr_res = np.around(rfr_res)

      model_metrics['RandomForestRegressor']['train_r2_score'] += rf_reg.score(x_train, y_train)
      model_metrics['RandomForestRegressor']['mean_absolute_error'] += mean_absolute_error(y_test, rfr_res)
      model_metrics['RandomForestRegressor']['mean_absolute_percentage_error'] += mean_absolute_percentage_error(y_test, rfr_res)
      model_metrics['RandomForestRegressor']['mae/mean'] += (mean_absolute_error(y_test, rfr_res))/mean(y_test)
      model_metrics['RandomForestRegressor']['mean_squared_error'] +=mean_squared_error(y_test, rfr_res)
      model_metrics['RandomForestRegressor']['RMSE'] += sqrt(mean_squared_error(y_test, rfr_res))
      model_metrics['RandomForestRegressor']['max_error'] += max_error(y_test, rfr_res)
      model_metrics['RandomForestRegressor']['r2_score'] += r2_score(y_test, rfr_res)

      rf_reg_feature_importances = pd.Series(rf_reg.feature_importances_, index=features)


      gb_reg = GradientBoostingRegressor(random_state=42, n_estimators=200)
      gb_reg.fit(x_train, y_train)
      gbr_res = gb_reg.predict(x_test)
      gbr_res = np.around(gbr_res)
      model_metrics['GradientBoostingRegressor']['train_r2_score'] += gb_reg.score(x_train, y_train)
      model_metrics['GradientBoostingRegressor']['mean_absolute_error'] += mean_absolute_error(y_test, gbr_res)
      model_metrics['GradientBoostingRegressor']['mean_absolute_percentage_error'] += mean_absolute_percentage_error(y_test, gbr_res)
      model_metrics['GradientBoostingRegressor']['mae/mean'] += mean_absolute_error(y_test, gbr_res)/mean(y_test)
      model_metrics['GradientBoostingRegressor']['mean_squared_error'] += mean_squared_error(y_test, gbr_res)
      model_metrics['GradientBoostingRegressor']['RMSE'] += sqrt(mean_squared_error(y_test, gbr_res))
      model_metrics['GradientBoostingRegressor']['max_error'] += max_error(y_test, gbr_res)
      model_metrics['GradientBoostingRegressor']['r2_score'] += r2_score(y_test, gbr_res)


      adb_reg = AdaBoostRegressor(random_state=42, n_estimators=200 )
      adb_reg.fit(x_train, y_train)
      adbr_res = adb_reg.predict(x_test)
      adbr_res = np.around(adbr_res)

      model_metrics['AdaBoostRegressor']['train_r2_score'] += adb_reg.score(x_train, y_train)
      model_metrics['AdaBoostRegressor']['mean_absolute_error'] += mean_absolute_error(y_test, adbr_res)
      model_metrics['AdaBoostRegressor']['mean_absolute_percentage_error'] += mean_absolute_percentage_error(y_test, adbr_res)
      model_metrics['AdaBoostRegressor']['mae/mean'] += mean_absolute_error(y_test, adbr_res)/mean(y_test)
      model_metrics['AdaBoostRegressor']['mean_squared_error'] +=mean_squared_error(y_test, adbr_res)
      model_metrics['AdaBoostRegressor']['RMSE'] += sqrt(mean_squared_error(y_test, adbr_res))
      model_metrics['AdaBoostRegressor']['max_error'] += max_error(y_test, adbr_res)
      model_metrics['AdaBoostRegressor']['r2_score'] += r2_score(y_test, adbr_res)

      svr_linear = SVR(kernel="linear", C=100, gamma=0.1, epsilon=0.1)
      svr_linear.fit(x_train, y_train)
      svr_linear_res = svr_linear.predict(x_test)
      svr_linear_res = np.around(svr_linear_res)

      model_metrics['SVR_linear']['train_r2_score'] += svr_linear.score(x_train, y_train)
      model_metrics['SVR_linear']['mean_absolute_error'] += mean_absolute_error(y_test, svr_linear_res)
      model_metrics['SVR_linear']['mean_absolute_percentage_error'] += mean_absolute_percentage_error(y_test, svr_linear_res)
      model_metrics['SVR_linear']['mae/mean'] += mean_absolute_error(y_test, svr_linear_res)/mean(y_test)
      model_metrics['SVR_linear']['mean_squared_error'] +=mean_squared_error(y_test, svr_linear_res)
      model_metrics['SVR_linear']['RMSE'] += sqrt(mean_squared_error(y_test, svr_linear_res))
      model_metrics['SVR_linear']['max_error'] += max_error(y_test, svr_linear_res)
      model_metrics['SVR_linear']['r2_score'] += r2_score(y_test, svr_linear_res)

  for model in models:
        for metric in metrics:
            model_metrics[model][metric] = model_metrics[model][metric]/10.0

  return model_metrics, rf_reg_feature_importances



#### Train the ML models for all locations

The training input are features from all locations

1. Aggregate features from all locations

In [7]:
bloom_features = dc_bloom_features.append(kyoto_bloom_features)
bloom_features = bloom_features.append(liestal_bloom_features)
bloom_features.reset_index(drop = True, inplace = True)

2. Train the model and evaluate the model by 10-fold cross-validation

In [8]:
all_loc_model_metrics, all_loc_rf_reg_feature_importances = pbd_pred_kfold(bloom_features, used_features)

##### Evaluation Metrics of ML models
Support Vector Regression with linear kernel performed the best with lowest values of Mean absolute error (3.63) and MSE (24.09)

In [9]:
all_loc_model_metrics_df = pd.DataFrame.from_dict(all_loc_model_metrics, orient='index')
all_loc_model_metrics_df.reset_index(inplace=True)
all_loc_model_metrics_df.rename(columns = {'index':'model'}, inplace = True)
all_loc_model_metrics_df

,model,train_r2_score,mean_absolute_error,mean_absolute_percentage_error,mae/mean,mean_squared_error,RMSE,max_error,r2_score
0,RandomForestRegressor,0.938879,4.200000,0.045079,0.044573,31.091026,5.172276,11.2,0.393033
1,GradientBoostingRegressor,0.999628,3.975641,0.042355,0.042172,30.335897,5.063414,10.5,0.386871
2,AdaBoostRegressor,0.881295,4.323718,0.046164,0.045896,31.435256,5.333416,11.2,0.346808
3,SVR_linear,0.733631,3.628205,0.038326,0.038425,24.093590,4.496378,9.1,0.553055


#####  Impurity-based feature importances (based on Random Forests Regression) 
The higher, the more important the feature. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance. 
Refer to [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)for more information

In [10]:
all_loc_rf_reg_feature_importances.sort_values(ascending = False)

spring_avg_tmax          0.411232
feb_avg_tmax             0.115118
spring_avg_daily_temp    0.099818
dec_avg_tmax             0.090479
spring_avg_tmin          0.051131
dec_avg_daily_temp       0.039214
jan_avg_tmax             0.027266
dec_avg_tmin             0.025835
feb_avg_daily_temp       0.023749
feb_avg_tmin             0.020032
winter_avg_tmax          0.017706
AGDD_Mar_13              0.017364
winter_avg_tmin          0.013529
winter_avg_daily_temp    0.011244
jan_avg_tmin             0.010555
jan_avg_daily_temp       0.009984
lat                      0.009479
alt                      0.006266
dtype: float64

#### Prediction for PBD in 2022

1. Load future weather features for all locations

In [11]:
future_weather_features_all_loc = pd.read_csv( '../../data/processed_data/features/future_weather_features_all_loc.csv', index_col = 0)
future_weather_features_all_loc

,location,lat,long,alt,year,AGDD_Mar_13,winter_avg_daily_temp,winter_avg_tmax,winter_avg_tmin,spring_avg_daily_temp,...,dec_avg_tmin,jan_avg_daily_temp,jan_avg_tmax,jan_avg_tmin,feb_avg_daily_temp,feb_avg_tmax,feb_avg_tmin,march_avg_daily_temp,march_avg_tmax,march_avg_tmin
0,washingtondc,38.885350,-77.038628,0,2022,322.375863,4.188278,9.700752,-1.324195,6.551500,...,1.999417,0.384800,5.057945,-4.288346,4.707168,11.136505,-1.722169,6.551500,10.245385,2.857615
1,kyoto,35.011983,135.676114,44,2022,837.736174,5.197681,7.442456,2.952907,6.484346,...,3.769405,4.264947,6.167337,2.362558,5.254737,7.339449,3.170025,6.484346,7.397692,5.571000
2,liestal,47.481400,7.730519,350,2022,324.987468,1.495270,4.397471,-1.406931,5.229115,...,-0.805586,0.529195,3.361908,-2.303519,2.785496,6.651046,-1.080054,5.229115,12.403538,-1.945308
3,vancouver,49.223692,-123.163625,24,2022,420.776887,4.188795,6.223602,2.153987,6.484346,...,0.032523,4.264947,6.167337,2.362558,5.254737,7.339449,3.170025,6.484346,7.397692,5.571000


2. Train the SVR model using historical data (1979 - 2021)
3. Predict the PBD in 2022 for 4 locations

In [12]:
def pbd_pred(train_data, future_data, features):
  x_train = train_data[features]
  y_train = train_data['bloom_doy']
  x_future = future_data[features]
  std_scaler = StandardScaler().fit(x_train)
  x_train = std_scaler.transform(x_train)
  x_future = std_scaler.transform(x_future)


  svr_linear = SVR(kernel="linear", C=100, gamma=0.1, epsilon=0.1)
  svr_linear.fit(x_train, y_train)
  svr_linear_res = svr_linear.predict(x_future)

  train_r2_score  = svr_linear.score(x_train, y_train)

  svr_linear_res = np.around(svr_linear_res)
  return svr_linear_res, train_r2_score


In [13]:
svr_predictions, train_r2_score  = pbd_pred(bloom_features, future_weather_features_all_loc, used_features)

In [14]:
train_r2_score

0.7291860421548921

5. Format the table for predictions

In [15]:
predictions = future_weather_features_all_loc.loc[:,['location','year']]
predictions['pred_bloom_doy']  = svr_predictions
predictions['pred_bloom_date'] =  predictions["year"]*1000 + predictions["pred_bloom_doy"]
predictions["pred_bloom_date"] = pd.to_datetime(predictions["pred_bloom_date"], format = "%Y%j")
predictions

,location,year,pred_bloom_doy,pred_bloom_date
0,washingtondc,2022,95.0,2022-04-05
1,kyoto,2022,102.0,2022-04-12
2,liestal,2022,86.0,2022-03-27
3,vancouver,2022,87.0,2022-03-28


In [16]:
locations = future_weather_features_all_loc['location'].unique().tolist()
format_cols = ['year'] + locations
predictions_formated = pd.DataFrame(columns = format_cols)
predictions_formated['year'] = [2022]
predictions_formated.set_index('year',inplace = True)
predictions_formated.loc[2022] = svr_predictions
# predictions_formated.reset_index(inplace = True)
predictions_formated

,washingtondc,kyoto,liestal,vancouver
year,,,,
2022,95.0,102.0,86.0,87.0
